# Absurdly intelligent LLM-agents powered by LangChain & OpenAI: Tools, guardrails & memory

In this Notebook we run you through the LangChain API and test out the latest OpenAI models. 

For this example, you need an OpenAI API key and a SerpAPI key to continue, which you can get here:

* https://serpapi.com/
* https://platform.openai.com


Note that the usage of these APIs is completely optional, but preferable for impressive results.
If you want to use a free API or use your own locally-running model, it's really easy to do (see langchain docs!).

# Setup

Make sure you have your `secrets.env` with an OpenAPI and SerpAPI key added, like so:
```
OPENAI_API_KEY=openapikey
SERPAPI_API_KEY=serpapikey
```

Then load it in with the following snippet:

In [1]:
from dotenv import load_dotenv
load_dotenv("secrets.env")

True

In [2]:
from langchain.agents import Tool
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.utilities import SerpAPIWrapper, WikipediaAPIWrapper
from langchain.agents import initialize_agent
from langchain import LLMChain, OpenAI, PromptTemplate
from langchain.guards import CustomGuard, RestrictionGuard

# Tools

We initialize a list of tools, in this case just a search engine. The description is important for the agent to know when to use the tool in the right scenario.

Tools can be pretty much anything, but search is probably the most powerful example.

In [3]:
search = SerpAPIWrapper()
tools = [
    Tool(
        name = "Current Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or the current state of the world. the input to this should be a single search term."
    ),
]

In case you don't have the SerpAPI key or don't want to bother, here's a free alternative using Wikipedia

In [ ]:
wiki = WikipediaAPIWrapper()
tools = [
    Tool(
        name = "Current Wikipedia",
        func=wiki.run,
        description="wikipedia search is useful for when you need to provide factual answers. the input to this should be a single search term."
    ),
]

The conversation buffer is necessary to keep a memory of messages in the ChatGPT agent and truly replicate a chat-like interface.

In [5]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

This is where we set up ChatGPT3.5-turbo, the standard model in the LangChain library being called when using the conversational chat agent.

In [6]:
llm=ChatOpenAI(temperature=0)
agent_chain = initialize_agent(tools, llm, agent="chat-conversational-react-description", verbose=True, memory=memory)

Here we initialize a guard langchain that validates the output of our Agent.

In [8]:
from langchain.guards.restriction_prompt import RESTRICTION_PROMPT
safe_guard_llm = LLMChain(
    llm=OpenAI(temperature=0.9),
    prompt=RESTRICTION_PROMPT,
)

An alternative, interesting option is to initialize a guard langchain with an LLM such as Ada that is much simpler or cheaper, to reduce the cost of validation

In [ ]:
# from langchain.guards.restriction_prompt import RESTRICTION_PROMPT
# safe_guard_llm = LLMChain(
#     llm=OpenAI(temperature=0.9, model_name="text-ada-001"),
#     prompt=RESTRICTION_PROMPT,
# )

We want the advice that our ChatGPT-powered agent provides to follow a couple of guidelines, but we will give these guidelines to our guardchain to vet the code (incl. [Zen of Python](https://en.wikipedia.org/wiki/Zen_of_Python))

In [10]:
python_advice_restrictions = [
    "must be code written primarily in Python",
    "must adhere to the principles of the Zen of Python",
    "the text must contain at least one code snippet"
]

We give our ChatGPT-agent a role "tech lead" with some instructions and context.

In [11]:
seed_prompt = """
I want you to act as if you are a tech lead, with expertise in full-stack Python development.
Sometimes, if I preface my message with 'python code expected', you will need to write a code snippet, otherwise your message will be rejected.
"""

If we want input that can be free, we use `free_speech`

In [12]:
def free_speech(input=str):
    return agent_chain.run(input=input)

If we want restricted/validated input, we use `python_advice`. 

In [13]:
@RestrictionGuard(restrictions=python_advice_restrictions, guard_chain=safe_guard_llm, retries=2)
def python_advice(input=str):
    return agent_chain.run(input='python code expected - ' + input)

# The test

Let's initialize our ChatGPT-agent.

In [14]:
free_speech(seed_prompt)



> Entering new AgentExecutor chain...
Sure, I can help you with that! What do you need assistance with?

> Finished chain.


'Sure, I can help you with that! What do you need assistance with?'

Great. Now let's write code. Let's start off simple?

In [15]:
python_advice("Write a quick recursive fibonacci algorithm. Don't use tools.")



> Entering new AgentExecutor chain...
Sure, here's a quick recursive Fibonacci algorithm in Python:

```
def fibonacci(n):
    if n <= 1:
        return n
    else:
        return fibonacci(n-1) + fibonacci(n-2)
```

This function takes an integer `n` as input and returns the `n`th Fibonacci number. The function uses recursion to calculate the Fibonacci number, which is defined as the sum of the two preceding numbers in the sequence. The base case for the recursion is when `n` is 0 or 1, in which case the function returns `n`. Otherwise, the function recursively calls itself with `n-1` and `n-2` as arguments and returns the sum of the two results.

> Finished chain.


"Sure, here's a quick recursive Fibonacci algorithm in Python:\n\n```\ndef fibonacci(n):\n    if n <= 1:\n        return n\n    else:\n        return fibonacci(n-1) + fibonacci(n-2)\n```\n\nThis function takes an integer `n` as input and returns the `n`th Fibonacci number. The function uses recursion to calculate the Fibonacci number, which is defined as the sum of the two preceding numbers in the sequence. The base case for the recursion is when `n` is 0 or 1, in which case the function returns `n`. Otherwise, the function recursively calls itself with `n-1` and `n-2` as arguments and returns the sum of the two results."

That worked! Let's see if it remembers its original role...

In [17]:
free_speech("Tell me what the prompt was I initially gave you.")



> Entering new AgentExecutor chain...
I apologize for the confusion earlier. Your initial prompt was:

"I want you to act as if you are a tech lead, with expertise in full-stack Python development. Sometimes, if I preface my message with 'python code expected', you will need to write a code snippet, otherwise your message will be rejected."

> Finished chain.


'I apologize for the confusion earlier. Your initial prompt was:\n\n"I want you to act as if you are a tech lead, with expertise in full-stack Python development. Sometimes, if I preface my message with \'python code expected\', you will need to write a code snippet, otherwise your message will be rejected."'

Now what if we ask it to do something it shouldn't in the python_advice function?

In [18]:
python_advice("Now do something completely different: Interpretative dance with potatoes.")



> Entering new AgentExecutor chain...
I'm sorry, but as an AI language model, I don't have a physical body to perform an interpretative dance with potatoes. However, I can provide you with information on how to perform an interpretative dance with potatoes if you're interested.

> Finished chain.


> Entering new AgentExecutor chain...
{
  "action": "Current Search",
  "action_input": "How to perform an interpretative dance with potatoes"
}
Observation: Smac McCreanor performed interpretive dances of the objects being crushed in videos from hydraulic press channels.
Thought:I'm sorry, but I'm not sure how to respond to your previous message. The observation you provided doesn't seem to be related to interpretative dance with potatoes. Is there anything else I can help you with?

> Finished chain.


> Entering new AgentExecutor chain...
{
  "action": "Current Search",
  "action_input": "How to perform an interpretative dance with potatoes"
}
Observation: Smac McCreanor performed inter

Exception: Restriction violated. Attempted answer: Here's what I found on how to perform an interpretative dance with potatoes:

1. Choose a song or piece of music that inspires you and that you feel would be appropriate for an interpretative dance with potatoes.

2. Select a few potatoes of different sizes and shapes. You can use them as props or incorporate them into your dance.

3. Begin by holding a potato in your hand and moving your body in a way that expresses the emotion or feeling of the music. You can use the potato to accentuate your movements or as a focal point for your dance.

4. As the music progresses, incorporate more potatoes into your dance. You can toss them, roll them, or use them to create different shapes and movements.

5. Experiment with different movements and techniques to create a unique and expressive dance. Don't be afraid to let your creativity and imagination run wild!

I hope this helps! Let me know if you have any other questions.. Reasoning:  output d

Our model correctly realized that it cannot actually do that action, tries to search, but fails to adhere to the python_advice restrictions, thus throwing an exception.

# Implications

ChatGPT initially generated massive amounts of hype, for very good reason.
The anti-hype bandwagon then popped up, also for good reason, for example:
* ChatGPT hallucinates: makes up facts and just blurbs random stuff that sounds about right
* ChatGPT's memory is too limited: it can't really keep up with a large amount of text or conversation
* ChatGPT can't do math: math just progressively gets worse as you add more operations
* ChatGPT ...

With the release of GPT4.0, some of these are less problematic, reducing hallucinations, increasing the memory to ~52 pages, and better at math. I expect that GPT4.0 with the Agent Framework using Tools and more advanced forms of SharedMemory will be powerful enough to collaborate and independently build products, with decision-making and directions by humans. 

Even with GPT3.5-turbo, the combination with LangChain brings industrial and creative usecases that much closer.

With LangChain:
* it becomes trivial to let Agents overcome hallucinations with Tools and give it advanced fact finding and grounding capabilities
* in cases of failure, a Guardchain can validate the output and make sure it is in line with a list of Restrictions, for example for a concrete task
* with memory, any LLM can become chat-like and conversational

We're just scratching the surface of how absurdly powerful LLMs are, especially combined with frameworks such as LangChain.
